In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmzReviews").getOrCreate()

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [ ]:
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True)
df.show(5)

### Get Number Of Reviews

In [ ]:
df.count()

### Create a Dataframe for Reviews schema

In [ ]:
df_review_id = df.select('review_id','customer_id','product_id','product_parent','review_date')
df_review_id.show(5)

### Create a dataframe for Products schema

In [ ]:
df_products = df.select('product_id','product_title').distinct()
df_products.show(5)

### Create a dataframe for Customers schema

In [ ]:
df_customer = df.groupBy("customer_id").count()

### Create a dataframe for Vine schema

In [ ]:
df_vine = df.select('review_id' , 'star_rating', 'helpful_votes', 'total_votes', 'vine')
df_vine.show(10)

### Create logic to determine if the Reviewer is a Vine or not

In [ ]:
df.createOrReplaceTempView("reviews")

In [ ]:
df_Vines = spark.sql("SELECT customer_id, count(1) as Num_Rev , sum(helpful_votes) as Num_Helpful_Rev, sum(total_votes) as Num_Total_Rev from reviews group by customer_id having Num_Helpful_Rev > (Num_Total_Rev / 2) and Num_Helpful_Rev > 0 and Num_Total_Rev > 1" )

In [ ]:
df_Vines.show()

In [ ]:
df_Vines.count()

In [ ]:
df_customer.count()

### Analysis - Inference
#### Out Of 2,561,107 Reviewers only 591,466 qualify to be a Vine.
#### A reviewer is Qualified as a Vine if there more than 50% of total reviews are helpful. 